# Experimento TCC

## Libs

In [1]:
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from multiprocessing import cpu_count

import pandas as pd
import numpy as np

In [2]:
from modules.classNCA import NCA
from modules.optimization_funcs import PSO, Gradient
from modules.utils import folds_run, sklearn_folds_run

In [38]:
dtypes = {
    'tau1': float, 'tau2': float, 'tau3': float, 'tau4': float,'p1': float, 'p2': float,
    'p3': float, 'p4': float, 'g1': float, 'g2': float, 'g3': float, 'g4': float}

EletricalGrid = pd.read_csv('dataset/Data_for_UCI_named.csv',
                            dtype=dtypes).drop('stab', axis=1)

In [4]:
X, y = EletricalGrid.drop('stabf',axis=1), EletricalGrid.stabf 

In [5]:
pso_options = {'optimization_func':PSO, 'max_iter':1000, 'k':100, 'cpu_count':None}#cpu_count())

In [7]:
results = folds_run(X, y, model_class=NCA, model_options=pso_options)
for i in results:
    print(f'{i} mean = {np.array(results[i]).mean()}')

5it [00:37,  7.45s/it]

acc mean = 0.8099999999999999
mcc mean = 0.5880783812539206


In [19]:
nca_sklearn = NeighborhoodComponentsAnalysis()
sklearn_options = {'n_components':None, 'max_iter':1000,'random_state':42}

In [37]:
npX, npy = np.asarray(X), np.asarray(y)
results = sklearn_folds_run(npX, npy, 100, model_class=NeighborhoodComponentsAnalysis, model_options=sklearn_options)
for i in results:
    print(f'{i} mean = {np.array(results[i]).mean()}')

5it [00:18,  3.61s/it]

acc mean = 0.859
mcc mean = 0.6954053683643082
